In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc
import glob
os.getcwd()



'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs,files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
prior_quarter_End=datetime.date(2018,5,5)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2018-05-05
recent_complete_quarter_End 2018-08-04
lapsed_date_begin 2014-04-29


In [4]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2014-08-26
2017-02-26


In [5]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922[dftrans_before_20180922['transaction_date']<=str(recent_complete_quarter_End)]
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-08-08 13:59:24.479712
2 2019-08-08 13:59:53.006778
3 2019-08-08 14:00:18.392202
4 2019-08-08 14:00:48.348124
5 2019-08-08 14:01:19.681932
6 2019-08-08 14:01:55.468552
7 2019-08-08 14:02:26.304199
8 2019-08-08 14:02:55.485992
9 2019-08-08 14:03:27.198322
10 2019-08-08 14:04:07.897440
11 2019-08-08 14:04:53.359118
12 2019-08-08 14:05:41.544961
13 2019-08-08 14:06:24.807616
14 2019-08-08 14:07:08.076326
15 2019-08-08 14:08:04.173499


In [6]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)

print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(119832143, 2)


196

In [7]:
weeks_2019Q2=[datetime.date(2019,5,11)+datetime.timedelta(days=x*7) for x in range(13)]
weeks_2018Q2=[x-datetime.timedelta(days=52*7) for x in weeks_2019Q2]

Q2_files_2018_hist=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
Q2_files_2018_hist=[x for x in Q2_files_2018_hist if x.split("ales_week_ending_")[1][:10] in [str(x) for x in weeks_2018Q2]]
print(len(Q2_files_2018_hist))

folder_2018_by_week="/home/jian/BigLots/2018_by_weeks/"

Q2_files_2018_all_types=list(recursive_file_gen(folder_2018_by_week))
Q2_files_2018_all_types=[x for x in Q2_files_2018_all_types if "aily" in x]
Q2_files_2018_all_types=[x for x in Q2_files_2018_all_types if ".txt" in x]

Q2_files_2018=[]
for week_end_date in weeks_2018Q2:
    file_path=[x for x in Q2_files_2018_all_types if str(week_end_date) in x]
    Q2_files_2018=Q2_files_2018+file_path
    
Q2_files_2018_hist=sorted(Q2_files_2018_hist)
Q2_files_2018=sorted(Q2_files_2018)
Q2_files_2018=Q2_files_2018_hist+Q2_files_2018
print(len(Q2_files_2018))
Q2_files_2018



5
13


['/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-19.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-26.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-02.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-09.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-14/MediaStormDailySales20180717-113630-767.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-21

In [8]:
for file in Q2_files_2018:
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['customer_id_hashed','transaction_dt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df.drop_duplicates()
    df=df.rename(columns={"transaction_dt":"transaction_date"})
    all_rewards_most_recent=all_rewards_most_recent.append(df)
    print(file,datetime.datetime.now())
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates()

/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt 2019-08-08 14:12:44.933949
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-19.txt 2019-08-08 14:13:06.832996
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-26.txt 2019-08-08 14:13:35.018863
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-02.txt 2019-08-08 14:14:00.426050
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-09.txt 2019-08-08 14:14:25.273749
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt 2019-08-08 14:14:50.390212
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt 2019-08-08 14:15:15.544308
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt 2019-08-08 14:15:44.785857
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/Medi

In [9]:
df_copy_before_2019Q2=all_rewards_most_recent[all_rewards_most_recent['transaction_date']<=str(prior_quarter_End)]
df_copy_before_2019Q2=df_copy_before_2019Q2.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
df_copy_before_2019Q2=df_copy_before_2019Q2.drop_duplicates(['customer_id_hashed'])
df_copy_before_2019Q2=df_copy_before_2019Q2.rename(columns={"transaction_date":"transaction_date_before_the_Quarter"})

all_rewards_most_recent_After=all_rewards_most_recent[all_rewards_most_recent['transaction_date']<=str(recent_complete_quarter_End)]
all_rewards_most_recent_After=all_rewards_most_recent_After[['customer_id_hashed','transaction_date']].sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(df_copy_before_2019Q2,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After
del df_copy_before_2019Q2
gc.collect()

243

In [10]:
prior_quarter_End

datetime.date(2018, 5, 5)

In [11]:
recent_complete_quarter_End

datetime.date(2018, 8, 4)

In [12]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent['transaction_date_before_the_Quarter']
# Filled the na before with a later adte 2099-12-31

all_rewards_most_recent['transaction_date_after_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_recent_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent['transaction_date_after_the_Quarter']
print(datetime.datetime.now())

2019-08-08 14:53:24.046571


In [13]:
all_rewards_most_recent['Month_to_pre_Quarter_End']=all_rewards_most_recent['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_recent_Quarter_End']=all_rewards_most_recent['Days_to_recent_Quarter_End'].apply(lambda x: x.days/(365.25/12))
print(datetime.datetime.now())

2019-08-08 15:01:07.395845


In [14]:
all_rewards_most_recent['Group_before_the_Quarter']=np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_the_Quarter']=np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [15]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Lapsed_13_48', 'Active', 'NotAvailable_Before_Quarter']

In [16]:
all_rewards_most_recent['Group_after_the_Quarter'].unique()

array(['Lapsed_13_48', 'Active'], dtype=object)

In [17]:
all_rewards_most_recent.shape

(22438352, 9)

In [18]:
all_rewards_most_recent['customer_id_hashed'].nunique()

22438352

In [19]:
weeks_2019Q2=[datetime.date(2019,5,11)+datetime.timedelta(days=x*7) for x in range(13)]
weeks_2018Q2=[x-datetime.timedelta(days=52*7) for x in weeks_2019Q2]

Q2_files_2018_hist=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
Q2_files_2018_hist=[x for x in Q2_files_2018_hist if x.split("ales_week_ending_")[1][:10] in [str(x) for x in weeks_2018Q2]]
print(len(Q2_files_2018_hist))

folder_2018_by_week="/home/jian/BigLots/2018_by_weeks/"

Q2_files_2018_all_types=list(recursive_file_gen(folder_2018_by_week))
Q2_files_2018_all_types=[x for x in Q2_files_2018_all_types if "aily" in x]
Q2_files_2018_all_types=[x for x in Q2_files_2018_all_types if ".txt" in x]

Q2_files_2018=[]
for week_end_date in weeks_2018Q2:
    file_path=[x for x in Q2_files_2018_all_types if str(week_end_date) in x]
    Q2_files_2018=Q2_files_2018+file_path
    
Q2_files_2018_hist=sorted(Q2_files_2018_hist)
Q2_files_2018=sorted(Q2_files_2018)
Q2_files_2018=Q2_files_2018_hist+Q2_files_2018
print(len(Q2_files_2018))
Q2_files_2018

5
13


['/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-19.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-26.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-02.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-09.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-14/MediaStormDailySales20180717-113630-767.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-21

In [20]:
RecentQuarter_sales_by_id=pd.DataFrame()

for file in Q2_files_2018:
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    print(df['transaction_dt'].max(),df['transaction_dt'].min())
    
    df_sales=df.groupby(['customer_id_hashed'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales_recent_Quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_recent_Quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
    
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"
RecentQuarter_sales_by_id.shape

2018-05-12 2018-05-06
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt 2019-08-08 15:02:54.365714
2018-05-19 2018-05-13
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-19.txt 2019-08-08 15:03:28.773870
2018-05-26 2018-05-20
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-26.txt 2019-08-08 15:04:07.667138
2018-06-02 2018-05-27
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-02.txt 2019-08-08 15:04:47.369618
2018-06-09 2018-06-03
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-09.txt 2019-08-08 15:05:25.619811
2018-06-16 2018-06-10
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt 2019-08-08 15:06:05.003978
2018-06-23 2018-06-17
/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt 2019-08-08 15:06:46.859369
2018-06-30 2

(7680086, 4)

In [21]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(22438352, 12)

In [22]:
new_sign_ups_2018Q2=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                  compression="zip",sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_2018Q2=new_sign_ups_2018Q2[new_sign_ups_2018Q2['sign_up_date']<=str(recent_complete_quarter_End)]
new_sign_ups_2018Q2=new_sign_ups_2018Q2[new_sign_ups_2018Q2['sign_up_date']>str(prior_quarter_End)]

new_sign_ups_2018Q2['NewRewards']="Recent_Quarter_New_Sign_Ups"
print(new_sign_ups_2018Q2['sign_up_date'].min())

new_sign_ups_2018Q2=new_sign_ups_2018Q2.drop_duplicates("customer_id_hashed")

2018-05-06


In [23]:
new_sign_ups_2018Q2['sign_up_date'].max()

'2018-08-04'

In [24]:
new_sign_ups_2018Q2['sign_up_date'].min()

'2018-05-06'

In [25]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,new_sign_ups_2018Q2,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)

In [26]:
df_test_QC=all_rewards_most_recent[(all_rewards_most_recent['Group_after_the_Quarter']=="Lapsed_13_48") & (all_rewards_most_recent['Recent_Quarter_Shopping_Group']=="Recent_Quarter_Shopped")]
print(df_test_QC.shape)

df_test_QC.head(4)

(0, 14)


,customer_id_hashed,transaction_date_before_the_Quarter,transaction_date_after_the_Quarter,Days_to_pre_Quarter_End,Days_to_recent_Quarter_End,Month_to_pre_Quarter_End,Month_to_recent_Quarter_End,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,trans_recent_Quarter,Recent_Quarter_Shopping_Group,sign_up_date,NewRewards


In [27]:
all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape


(22725201, 14)

In [28]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q2_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()

output=pd.merge(output_id_count,output_id_Q2_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")
output


,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter,trans_recent_Quarter
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,8149614,0.000000e+00,0.0
1,Active,Active,Old_Rewards_Members,Recent_Quarter_Shopped,5705414,4.886251e+08,13963573.0
2,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,39,0.000000e+00,0.0
3,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,181,3.504347e+04,550.0
4,Active,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,1423693,0.000000e+00,0.0
5,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,294,0.000000e+00,0.0
6,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_Shopped,467614,3.403724e+07,709042.0
7,Lapsed_13_48,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,5182957,0.000000e+00,0.0
8,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,485,0.000000e+00,0.0
9,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_Shopped,490791,4.039950e+07,733908.0


In [29]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAvailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [30]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2018Q2_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [31]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2018Q2_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [ ]:
all_rewards_most_recent.shape

In [ ]:
all_rewards_most_recent['customer_id_hashed'].nunique()

In [29]:
all_rewards_most_recent[all_rewards_most_recent['sales_recent_Quarter']>0].shape

(7635879, 14)